In [1]:
import numpy as np
import pandas as pd
import json
import os
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=False)

<IPython.core.display.Javascript object>

In [2]:
def load_data(data_dir: str):
    gs_profile_file = os.path.join(data_dir, 'gs_profiles_fix.jsonl')
    data = []
    with open(gs_profile_file, 'r', encoding='utf8') as fp:
        for line in fp:
            data.append(json.loads(line))
    return data

def get_cited(text: str):
    line = text.splitlines()[1]
    _, c_all, c_5yr = line.split()
    return int(c_all), int(c_5yr)

In [7]:
data = load_data('../out')

In [8]:
columns  = ['name', 'homepage', 'cited_all', 'cited_5yr', 'cited_5yr_ratio', 'tags', 'articles', 'brief']
rows = []
for obj in data:
    url = 'https://scholar.google.com' + obj['url']
    name = f'<a href="{url}">{obj["name"]}</a>'
    homepage = obj['homepage']
    if homepage:
        homepage = f' <a href="{homepage}">open</a>'
    c_all, c_5yr = get_cited(obj['cited_stats'])
    c_5yr_ratio = c_5yr / c_all if c_all > 0 else 0
    brief = obj['brief'].split('\n', 1)[1].lower().replace('\n', '<br>')
    email = 
    articles = '<br>'.join(obj['articles'])
    tags = '<br>'.join(obj['tags'])

    row = (name, homepage, c_all, c_5yr, c_5yr_ratio, tags, articles, brief)
    rows.append(row)

df = pd.DataFrame(rows, columns=columns)
show(df, 
     authWidth=False,
     style="table-layout:auto;width:100%;",
    )
